# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

ImportError: cannot import name 'gkey'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
df = pd.read_csv("./output_data/cities.csv")
df.head()

,City_ID,City,Cloudiness,Country,Date,Unnamed: 5,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,5/20/2019,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,5/20/2019,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,5/20/2019,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,5/20/2019,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,5/20/2019,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps library with API key - creates the base map
gmaps.configure(api_key=g_key)

NameError: name 'g_key' is not defined

In [ ]:
# Sort weatherpy df to columns lat, long, humidity

In [7]:
# Create a humidity Heatmap layer

locations=df[["Lat, Lng"]]
weights=df["Humidity"]
fig=gmaps.figure()
gmaps.heatmap_layer(locations, weights,dissipating=False, max_intensity=100, point_radius = 1)

fig

KeyError: "None of [Index(['Lat, Lng'], dtype='object')] are in the [columns]"

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#hotel_df=df.dropna(how="any")

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#hotel_df["Hotel Name"]=""

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

# # Create bank symbol layer
# bank_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
# )


# fig = gmaps.figure()
# fig.add_layer(bank_layer)

# fig
# Display figure
